# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [63]:
# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import ElasticNet
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor


%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

df = df.drop(columns=['data_ref', 'Unnamed: 0', 'id_cliente'])


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [38]:
# 1)

df = df.dropna()

df_dummies = pd.get_dummies(df, drop_first=True)

X = df_dummies.drop(columns=["renda"])
y = df_dummies["renda"]

# Separar dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#X_train_const = sm.add_constant(X_train)
#X_test_const = sm.add_constant(X_test)

bool_columns = X_train.select_dtypes(include=["bool"]).columns
bool_columns2 = X_test.select_dtypes(include=["bool"]).columns

# Converter as colunas booleanas para inteiros
X_train[bool_columns] = X_train[bool_columns].astype(int)
X_test[bool_columns2] = X_test[bool_columns2].astype(int)


In [32]:
# 2)

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Lista para armazenar resultados
results = []

# Loop para testar cada valor de alpha
for alpha in alphas:
    # Criação e treinamento do modelo Ridge
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train_const, y_train)
    
    # Predição na base de teste
    y_pred = ridge_model.predict(X_test_const)
    
    # Cálculo do R²
    r2 = r2_score(y_test, y_pred)
    
    # Armazena o resultado
    results.append((alpha, r2))

# Identifica o melhor modelo (maior R²)
best_alpha, best_r2 = max(results, key=lambda x: x[1])

# Exibição dos resultados
print("Resultados para cada alpha:")
for alpha, r2 in results:
    print(f"Alpha: {alpha:.3f}, R²: {r2:.4f}")

print("\nMelhor modelo:")
print(f"Alpha: {best_alpha:.3f}, R²: {best_r2:.4f}")


Resultados para cada alpha:
Alpha: 0.000, R²: 0.2980
Alpha: 0.001, R²: 0.2980
Alpha: 0.005, R²: 0.2980
Alpha: 0.010, R²: 0.2980
Alpha: 0.050, R²: 0.2980
Alpha: 0.100, R²: 0.2980

Melhor modelo:
Alpha: 0.100, R²: 0.2980


In [33]:
# 3)

# Lista de valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Lista para armazenar os resultados
results_lasso = []

# Loop para LASSO
for alpha in alphas:
    lasso_model = Lasso(alpha=alpha, max_iter=10000)  # max_iter aumentado para garantir convergência
    lasso_model.fit(X_train, y_train)
    y_pred_lasso = lasso_model.predict(X_test)
    r2_lasso = r2_score(y_test, y_pred_lasso)
    results_lasso.append((alpha, r2_lasso))

# Melhor modelo LASSO
best_alpha_lasso, best_r2_lasso = max(results_lasso, key=lambda x: x[1])

# Exibição dos resultados
print("Resultados para cada alpha (LASSO):")
for alpha, r2 in results_lasso:
    print(f"Alpha: {alpha:.3f}, R²: {r2:.4f}")

print("\nMelhor modelo LASSO:")
print(f"Alpha: {best_alpha_lasso:.3f}, R²: {best_r2_lasso:.4f}")

C:\Users\valdi\anaconda3\Lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\valdi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\valdi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.036e+11, tolerance: 8.060e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

Resultados para cada alpha (LASSO):
Alpha: 0.000, R²: 0.2980
Alpha: 0.001, R²: 0.2980
Alpha: 0.005, R²: 0.2980
Alpha: 0.010, R²: 0.2980
Alpha: 0.050, R²: 0.2980
Alpha: 0.100, R²: 0.2980

Melhor modelo LASSO:
Alpha: 0.100, R²: 0.2980


## Possíveis razões para resultados idênticos:

- Alpha baixo: Os valores de alpha testados (0,0.001,0.005,0.01,0.05,0.10,0.001,0.005,0.01,0.05,0.1) podem ser pequenos demais para que o LASSO tenha impacto, especialmente se os coeficientes ainda não foram reduzidos significativamente.
- Escala dos dados: Se os dados não estão padronizados (média 0 e desvio padrão 1), o efeito do alpha pode ser mascarado. Ridge e LASSO são sensíveis à escala das variáveis.
- Natureza dos dados: Se os dados têm poucas variáveis irrelevantes, o LASSO pode não estar zerando coeficientes, tornando os resultados similares aos do Ridge.

In [52]:
# 4)

def stepwise_selection(X_train, y_train, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS """
    
    included = list(initial_list)
    
    while True:
        changed=False
        
        # Forward step: Seleciona as melhores variáveis para incluir
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        
        for new_column in excluded:
            model = sm.OLS(y_train, sm.add_constant(X_train[included + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        
        best_pval = new_pval.min()
        
        # Se a variável a ser incluída tem p-value menor que o limiar de inclusão
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Add  {best_feature:30} with p-value {best_pval:.6f}')

        # Backward step: Exclui as variáveis com maior p-value (acima de threshold_out)
        if len(included) > 0:
            model = sm.OLS(y_train, sm.add_constant(X_train[included])).fit()
            pvalues = model.pvalues.iloc[1:]  # Ignora o intercepto
            worst_pval = pvalues.max()  # Variável com maior p-value
            
            if worst_pval > threshold_out:
                changed = True
                worst_feature = pvalues.argmax()
                included.remove(worst_feature)
                if verbose:
                    print(f'Drop {worst_feature:30} with p-value {worst_pval:.6f}')

        # Se não houve mudança (nenhuma inclusão ou exclusão), o processo termina
        if not changed:
            break
    
    return included

# Dados de exemplo (você deve substituir por seus dados reais)
# Certifique-se de que X e y estão prontos para a modelagem
# X é o DataFrame de variáveis independentes
# y é a variável dependente
variaveis = stepwise_selection(X_train, y_train)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.000000
Add  sexo_M                         with p-value 0.000000
Add  tipo_renda_Empresário          with p-value 0.000004
Add  idade                          with p-value 0.000024
Add  educacao_Superior completo     with p-value 0.000804
Add  posse_de_imovel                with p-value 0.043736
resulting features:
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'posse_de_imovel']


In [59]:
relevant_features = ['sexo_M', 'tempo_emprego', 'tipo_renda_Empresário', 
                     'idade', 'educacao_Superior completo', 'posse_de_imovel']

# Criar subconjuntos dos dados
X_relevant = X_train[relevant_features]
y_train_log = np.log(y_train)

# Ajustar o modelo OLS
#ols_model = sm.OLS(y_train, X_relevant).fit()
ols_model_log = sm.OLS(y_train_log, X_relevant).fit()

# Resumo do modelo
print(ols_model.summary())

                                 OLS Regression Results                                
Dep. Variable:                  renda   R-squared (uncentered):                   0.474
Model:                            OLS   Adj. R-squared (uncentered):              0.474
Method:                 Least Squares   F-statistic:                              1398.
Date:                Fri, 03 Jan 2025   Prob (F-statistic):                        0.00
Time:                        22:20:44   Log-Likelihood:                         -97098.
No. Observations:                9320   AIC:                                  1.942e+05
Df Residuals:                    9314   BIC:                                  1.943e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

In [62]:
X_relevant_test = X_test[relevant_features]
y_pred = ols_model.predict(X_relevant_test)


# R² (coeficiente de determinação)
r2 = r2_score(y_test, y_pred)

print(f'R-squared on Test Set: {r2:.4f}')

# Opcional: Exibir as primeiras previsões comparadas com os valores reais
print("\nFirst 10 predicted vs real values:")
print(pd.DataFrame({'Predicted': y_pred[:10], 'Real': y_test[:10]}))

R-squared on Test Set: 0.2926

First 10 predicted vs real values:
          Predicted     Real
14812   6231.557792  1705.55
11591   6379.142935  1748.99
13436    330.539309  1733.67
14948   7255.872333  2378.25
14509   3899.387804  1957.87
2955    2562.385207  1218.39
7148    -663.873294  2085.39
1337   10454.594734  8849.50
14688    634.940719  4676.05
10501    766.702786  1915.48


## Conclusão:
- Modelo OLS foi mais interpretável, pois usou apenas as variáveis com p-values significativos, mas possivelmente sofreu de overfitting ao se ajustar excessivamente aos dados de treinamento. Esse modelo pode ter um desempenho satisfatório em termos de R², mas provavelmente teve dificuldades em generalizar para o conjunto de teste.

- Ridge e Lasso apresentaram um R² mais baixo, mas sua capacidade de regularização ajudou a controlar o overfitting e a melhorar a generalização. Em situações com muitas variáveis e potencial multicolinearidade, esses modelos tendem a ser mais robustos e a ter um desempenho mais consistente em dados não vistos.

In [65]:
# Ajustar o modelo de árvore de regressão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_relevant, y_train)

# Fazer previsões no conjunto de teste
y_pred_tree = tree_model.predict(X_relevant_test)

# Calcular o R²
r2_tree = r2_score(y_test, y_pred_tree)

# Exibir o R² do modelo de árvore de regressão
print(f'R² da árvore de regressão: {r2_tree:.4f}')

R² da árvore de regressão: 0.2171
